## Perceptron

In [ ]:
import torch
import torch.nn as nn

class Perceptron(nn.Module):
    def __init__(self, input_size, output_size):
        super(Perceptron, self).__init__()
        self.input_size = input_size
        self.output_size = output_size

        # Initialize weights and biases for Linear layer
        self.W = nn.Parameter(torch.randn(output_size, input_size) * 0.01)
        self.b = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        # Apply the linear layer
        x = torch.matmul(x, self.W.T) + self.b
        return x


## Multi-layer perceptron

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases for the first Linear layer
        self.W_ih = nn.Parameter(torch.randn(hidden_size, input_size) * 0.01)
        self.b_ih = nn.Parameter(torch.zeros(hidden_size))

        # Initialize weights and biases for the second Linear layer
        self.W_ho = nn.Parameter(torch.randn(output_size, hidden_size) * 0.01)
        self.b_ho = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        # Apply the first linear layer with ReLU activation
        h = torch.relu(torch.matmul(x, self.W_ih.T) + self.b_ih)

        # Apply the second linear layer
        x = torch.matmul(h, self.W_ho.T) + self.b_ho
        return x


In [ ]:
## Recurrent Neural Network

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases for RNN
        self.W_ih = nn.Parameter(torch.randn(hidden_size, input_size) * 0.01)
        self.W_hh = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
        self.b_h = nn.Parameter(torch.zeros(hidden_size))

        # Initialize weights and biases for Linear layer
        self.W = nn.Parameter(torch.randn(output_size, hidden_size) * 0.01)
        self.b = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        seq_len, batch_size, _ = x.size()
        h = torch.zeros(batch_size, self.hidden_size, device=x.device)  # Initial hidden state

        # Apply the RNN step by step
        outputs = []
        for t in range(seq_len):
            x_t = x[t, :, :]
            h = nn.functional.relu(torch.matmul(x_t, self.W_ih.T) + torch.matmul(h, self.W_hh.T) + self.b_h)
            outputs.append(h.unsqueeze(0))

        out = torch.cat(outputs, dim=0)

        # Apply the linear layer
        x = torch.matmul(out, self.W.T) + self.b
        return x, out